In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import math
import time

In [79]:
# Fixed Parameters
S0 = 100
K = 80
k = math.log(K)
r = 0.05
q = 0.01
T = 1

In [3]:
# Parameters for FFT 
n = 12
N = 2 ** n

# Step-size
eta = 0.25

# Damping factor
alpha = 1.5

### $\lambda = \frac{2 \pi}{N \eta}$

In [ ]:
# Step-size in log strike space
lamda = (2 * math.pi / N) / eta

### Two commom choices for beta:
Middle of the range corresponds to at-the-money: set $\beta = ln(S_0) - \frac{N}{2}\lambda $

The first call corresponds to a specific strike $K$: set $\beta = ln(K)$

In [4]:
# Choice of beta
# beta = np.log(S0) - N * lamda / 2
beta = np.log(K)

### GeometricBrownianMotion
### $\phi(\nu) = e^{i(lnS_0\;+\;(r\;-\;q\;-\;\frac{\sigma^2}{2})T)\nu\;-\;\frac{\sigma^2 \nu^2}{2}T}$

In [180]:
class GeometricBrownianMotion:
    def __init__(self):
        pass
    
    def _characteristic_function(self, u, sig, S0, r, q, T):
        mu = np.log(S0) + (r - q - sig ** 2 / 2) * T
        a = sig * np.sqrt(T)
        phi = np.exp(1j * mu * u - (a * u) ** 2 / 2)
        
        return phi
    
    def evaluate_integral(self, sigma, S0, K, r, q, T, alpha, eta):
        discount_factor = math.exp(-r*T)

        sum1 = 0
        for j in range(N):
            nuJ = j * eta
            u = nuJ - (alpha + 1) * 1j
            psi_nuJ = discount_factor * self._characteristic_function(u, sigma, S0, r, q, T) / ((alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ))

            if j == 0:
                wJ = eta / 2
            else:
                wJ = eta

            sum1 += np.exp(-1j * nuJ * k) * psi_nuJ * wJ

        cT_k = (np.exp(-alpha * k) / math.pi) * sum1

        return np.real(cT_k)
    
    def fast_fourier_transform(self, sigma, S0, K, r, q, T, alpha, eta, n):
        N = 2 ** n

        # step-size in log strike space
        lda = (2 * np.pi / N) / eta

        #Choice of beta
        #beta = np.log(S0)-N*lda/2
        beta = np.log(K)

        # forming vector x and strikes km for m=1,...,N
        km = np.zeros((N))
        xX = np.zeros((N))

        discount_factor = math.exp(-r * T)

        nuJ = np.arange(N) * eta
        u = nuJ - (alpha + 1) * 1j
        psi_nuJ = self._characteristic_function(u, sigma, S0, r, q, T) / ((alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ))

        for j in range(N):  
            km[j] = beta + j * lda
            if j == 0:
                wJ = eta / 2
            else:
                wJ = eta

            xX[j] = np.exp(-1j * beta * nuJ[j]) * discount_factor * psi_nuJ[j] * wJ

        yY = np.fft.fft(xX)
        cT_km = np.zeros((N))  
        for i in range(N):
            multiplier = math.exp(-alpha * km[i]) / math.pi
            cT_km[i] = multiplier * np.real(yY[i])
        
        cT_k = cT_km[0]
        
        return cT_k


In [177]:
sigma = 0.30

In [182]:
gbm = GeometricBrownianMotion()
gbm.evaluate_integral(sigma, S0, K, r, q, T, alpha, eta)

25.614621075647143

In [183]:
gbm.fast_fourier_transform(sigma, S0, K, r, q, T, alpha, eta, n)

25.614621075647126

### Heston Stochastic Volitility Model
### $\phi(u) = \frac{exp\{ iu\;lnS_0\;+\;iu(r\;-\;q)T + \frac{\kappa \theta T(\kappa\;-\;i\rho \sigma u)}{\sigma^2} \}}{(cosh\frac{\gamma T}{2} + \frac{\kappa\;-\;i \rho \sigma u}{\gamma} sinh\frac{\gamma T}{2})^{\frac{2 \kappa \theta}{\sigma^2}}} \times exp\{ \frac{-(u^2\;+\;iu)v_0}{\gamma coth \frac{\gamma T}{2}\;+\;\kappa\;-\;i\rho \sigma u} \}$

In [246]:
class HestonStochasticVolitility:
    def __init__(self):
        pass
    
    def _characteristic_function(self, u, kappa, theta, sigma, rho, v0, S0, T):
        tmp = (kappa - 1j * rho * sigma * u)
        g = np.sqrt((sigma ** 2) * (u ** 2 + 1j * u) + tmp ** 2)
        
        pow1 = 2 * kappa * theta / (sigma ** 2)
        
        numer1 = (kappa * theta * T * tmp) / (sigma ** 2) + 1j * u * T * r + 1j * u * math.log(S0)
        log_denum1 = pow1 * np.log(np.cosh(g * T / 2) + (tmp / g) * np.sinh(g * T / 2))
        tmp2 = ((u * u + 1j * u) * v0) / (g / np.tanh(g * T / 2) + tmp)
        log_phi = numer1 - log_denum1 - tmp2
        phi = np.exp(log_phi)
        
        return phi
    
    def evaluate_integral(self, sigma, nu, theta, S0, K, r, q, T, alpha, eta):
        discount_factor = math.exp(-r*T)

        sum1 = 0
        for j in range(N):
            nuJ = j * eta
            u = nuJ - (alpha + 1) * 1j
            psi_nuJ = discount_factor * self._characteristic_function(u, kappa, theta, sigma, rho, v0, S0, T) / ((alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ))
            
            if j == 0:
                wJ = eta / 2
            else:
                wJ = eta

            sum1 += np.exp(-1j * nuJ * k) * psi_nuJ * wJ

        cT_k = (np.exp(-alpha * k) / math.pi) * sum1

        return np.real(cT_k)
    
    def fast_fourier_transform(self, sigma, nu, theta, S0, K, r, q, T, alpha, eta):
        N = 2 ** n

        # step-size in log strike space
        lda = (2 * np.pi / N) / eta

        #Choice of beta
        #beta = np.log(S0) - N * lda / 2
        beta = np.log(K)

        # forming vector x and strikes km for m=1,...,N
        km = np.zeros((N))
        xX = np.zeros((N))

        discount_factor = math.exp(-r * T)

        nuJ = np.arange(N) * eta
        u = nuJ - (alpha + 1) * 1j
        psi_nuJ = self._characteristic_function(u, kappa, theta, sigma, rho, v0, S0, T) / ((alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ))

        for j in range(N):  
            km[j] = beta + j * lda
            if j == 0:
                wJ = eta / 2
            else:
                wJ = eta

            xX[j] = np.exp(-1j * beta * nuJ[j]) * discount_factor * psi_nuJ[j] * wJ

        yY = np.fft.fft(xX)
        cT_km = np.zeros((N))  
        for i in range(N):
            multiplier = math.exp(-alpha * km[i]) / math.pi
            cT_km[i] = multiplier * np.real(yY[i])

        cT_k = cT_km[0]
        
        return cT_k
    

In [247]:
kappa = 2.0
theta = 0.05
sigma = 0.30
rho = -0.70
v0 = 0.04
u = 0 - (alpha + 1) * 1j
T = 1

In [248]:
hsv = HestonStochasticVolitility()
hsv.evaluate_integral(sigma, nu, theta, S0, K, r, q, T, alpha, eta)

25.242801609252524

In [249]:
hsv.fast_fourier_transform(sigma, nu, theta, S0, K, r, q, T, alpha, eta)

25.242801609252545

### Variance Gamma
### $\phi(u) = exp(iu(ln S_0\;+\;(r\;-\;q)T)) (\frac{1}{1\;-\;iu\theta \nu\;+\;\sigma^2u^2 \nu / 2})^{\frac{T}{\nu}}$

In [250]:
class VarianceGamma:
    def __init__(self):
        pass
    
    def _characteristic_function(self, u, sigma, nu, theta, S0, r, q, T):
        if nu == 0:
            mu = math.log(S0) + (r - q - theta - 0.5 * sigma ** 2) * T
            phi  = math.exp(1j * u * mu) * math.exp((1j * theta * u - 0.5 * sigma ** 2 * u ** 2) * T)
        else:
            mu  = math.log(S0) + (r - q + math.log(1 - theta * nu - 0.5 * sigma ** 2 * nu) / nu) * T
            phi = np.exp(1j * u * mu) * ((1 - 1j * nu * theta * u + 0.5 * nu * sigma ** 2 * u ** 2) ** (-T / nu))

        return phi
    
    def evaluate_integral(self, sigma, nu, theta, S0, K, r, q, T, alpha, eta):
        discount_factor = math.exp(-r * T)

        sum1 = 0
        for j in range(N):
            nuJ = j * eta
            u = nuJ - (alpha + 1) * 1j
            psi_nuJ = discount_factor * self._characteristic_function(u, sigma, nu, theta, S0, r, q, T) / ((alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ))
            
            if j == 0:
                wJ = eta / 2
            else:
                wJ = eta

            sum1 += np.exp(-1j * nuJ * k) * psi_nuJ * wJ

        cT_k = (np.exp(-alpha * k) / math.pi) * sum1

        return np.real(cT_k)
    
    def fast_fourier_transform(self, sigma, nu, theta, S0, K, r, q, T, alpha, eta):
        N = 2 ** n

        # step-size in log strike space
        lda = (2 * np.pi / N) / eta

        #Choice of beta
        #beta = np.log(S0)-N*lda/2
        beta = np.log(K)

        # forming vector x and strikes km for m=1,...,N
        km = np.zeros((N))
        xX = np.zeros((N))

        discount_factor = math.exp(-r * T)

        nuJ = np.arange(N) * eta
        u = nuJ - (alpha + 1) * 1j

        psi_nuJ = self._characteristic_function(u, sigma, nu, theta, S0, r, q, T) / ((alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ))

        for j in range(N):  
            km[j] = beta + j * lda
            if j == 0:
                wJ = eta / 2
            else:
                wJ = eta

            xX[j] = np.exp(-1j * beta * nuJ[j]) * discount_factor * psi_nuJ[j] * wJ

        yY = np.fft.fft(xX)
        cT_km = np.zeros((N))  
        for i in range(N):
            multiplier = math.exp(-alpha * km[i]) / math.pi
            cT_km[i] = multiplier * np.real(yY[i])

        cT_k = cT_km[0]
        
        return cT_k


In [251]:
sigma = 0.3
nu = 0.5
theta = -0.4

In [252]:
vg = VarianceGamma()
vg.evaluate_integral(sigma, nu, theta, S0, K, r, q, T, alpha, eta)

28.220281720017248

In [253]:
vg.fast_fourier_transform(sigma, nu, theta, S0, K, r, q, T, alpha, eta)

28.220281720017212